In [2]:
import sys
sys.path.insert(1, './scripts')

from libraries import *
from functions import *
from fitting import *
from util import *

%matplotlib qt
data[0]

/home/alanyu/anaconda3/lib/python3.8/site-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


Current progress: 50000
Current progress: 100000
Current progress: 150000
Current progress: 200000
Current progress: 250000
DONE


{'name': 'V1649180633_1.ir_rc20final.cub',
 'sequenceid': 'S59',
 'observationid': 'VIMS_129TI_REGMAP001_ISS',
 'latitude': 6.527894053385,
 'longitude': 117.23227585525,
 'phase': 4.35186034428,
 'incidence': 71.368914431631,
 'emission': 72.074559422824,
 'distance': 25208.69808185,
 'lineres': 38679.377390385,
 'sampres': 12885.433750399,
 'scposx': 15904.022205055,
 'scposy': 19558.581460496,
 'scposz': 49.274092100503,
 'sunposx': 993649050.83181,
 'sunposy': 1013825175.7289,
 'sunposz': 93586120.333315,
 'ettime': 323758866.57842,
 'starttime': '2010-04-05 16:59:19.48',
 'exposureduration': 220.0,
 'spectrum': array([ 0.0763505 ,  0.113136  ,  0.138922  ,  0.146524  ,  0.126     ,
         0.0936954 ,  0.0795553 ,  0.0763359 ,  0.0885651 ,  0.111927  ,
         0.131274  ,  0.140332  ,  0.13276   ,  0.100027  ,  0.0593369 ,
         0.0431969 ,  0.0295527 ,  0.0313845 ,  0.042422  ,  0.0622391 ,
         0.0827431 ,  0.0844537 ,  0.0981416 ,  0.109633  ,  0.124524  ,
         0.0

# Plot result from pickles

Given an index and a channel, plot the spectra of the channel along with the fitted functions

In [3]:
def plot_result(entry, channel):
    entry = data[entry]
    #print(entry['regular_fit']['powerlaw'])
    fig, axs = plt.subplots(2, 2)
    #print('powerlaw' in entry)
    if 'powerlaw' in entry['regular_fit']:
        axs[0,0].plot(vims_wave, entry['spectrum']-powerlaw_no_constant(vims_wave, *entry['regular_fit']['powerlaw'].values()), label = "Spectra (powerlaw adjusted)")
        axs[0,0].plot(vims_wave, gaussian(vims_wave, *entry['regular_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[0,0].plot(vims_wave, skew(vims_wave, *(entry['regular_fit'][channel]['skew'][x] for x in entry['regular_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
        
    else:
        axs[0,0].plot(vims_wave, entry['spectrum'], label = "Spectrum (no powerlaw adjustment)")
        axs[0,0].plot(vims_wave, gaussian(vims_wave, *entry['regular_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[0,0].plot(vims_wave, skew(vims_wave, *(entry['regular_fit'][channel]['skew'][x] for x in entry['regular_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
    
    axs[0,1].plot(vims_wave, entry['spectrum'], label="raw")
    axs[0,1].plot(vims_wave, entry['stefan_spectrum'], label = "Stefan")
    axs[0,1].plot(vims_wave, gaussian(vims_wave, *entry['stefan_fit'][channel]['gaussian'].values()), label = "Gaussian")
    axs[0,1].plot(vims_wave, skew(vims_wave, *(entry['stefan_fit'][channel]['skew'][x] for x in entry['stefan_fit'][channel]['skew'] if 'peak' not in x)), label = "Skew")
    
    if 'powerlaw' in entry['stefan-plaw_fit']:
        axs[1,0].plot(vims_wave, entry['spectrum'],label = 'raw')

        axs[1,0].plot(vims_wave, entry['stefan_spectrum']-powerlaw_no_constant(vims_wave, *entry['stefan-plaw_fit']['powerlaw'].values()), label = "Stefan spectra (powerlaw adjusted)")
        axs[1,0].plot(vims_wave, gaussian(vims_wave, *entry['stefan-plaw_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[1,0].plot(vims_wave, skew(vims_wave, *(entry['stefan-plaw_fit'][channel]['skew'][x] for x in entry['stefan-plaw_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
        
    else:
        axs[1,0].plot(vims_wave, entry['stefan_spectrum'], label = "Stefan spectrum (no powerlaw adjustment)")
        axs[1,0].plot(vims_wave, gaussian(vims_wave, *entry['stefan-plaw_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[1,0].plot(vims_wave, skew(vims_wave, *(entry['stefan-plaw_fit'][channel]['skew'][x] for x in entry['stefan-plaw_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
    
    
    axs[0,0].legend()
    axs[0,1].legend()
    axs[1,0].legend()
    
    plt.show()

plot_result(0,'2.0')

In [22]:
belet = glob.glob("new_pixels/belet/*.pkl")
belet[0].split('/')

['new_pixels',
 'belet',
 'belet_V1649180633_1.ir_rc20final_6.527894053385_117.23227585525.pkl']

### Test stefan second order correction

In [16]:
def stefan_second_test(entry, c):
    entry = data[entry]['stefan_spectrum']
    wings = {'2.0': (65,76),
            '1.2': (15, 27),
            '1.6': (37, 47),
            '1.0': (9, 16)}
    
    k = {'2.0': 1.29,
        '1.2': 1.50,
        '1.6': 1.6,
        '1.0': 1.15}

    new = np.array(entry)
    
    for channel in wings:
        adjust = k[channel]*(entry[wings[channel][0]] + entry[wings[channel][1]])/2
        new[wings[channel][0]:wings[channel][1]] -= adjust
        
    #plt.plot(vims_wave,entry, label = 'Stefan')
    #plt.plot(vims_wave,new, label = 'Stefan Second Order')
    
    #print(np.max(new))
    fits = fit(new, False, {'1.2':(15,27), '1.6':(37,47),'2.0':(65,76),'1.0':(9,16)})
    #fits = fit(new, False, {'1.2':(15,27), '1.6':(32,55),'2.0':(60,85),'1.0':(7,18)})
    #print(fits)
    #return fits
    #if fits:
    #    plt.plot(vims_wave, gaussian(vims_wave, 
    #                             fits[c]['gaussian']['mean'], 
    #                             fits[c]['gaussian']['sigma'], 
    #                             fits[c]['gaussian']['amp']), label = 'Gaussian')
    #plt.legend()
    
    #plt.show()
    
    return fits

L = []
good = 0
for i in range(100):
    fits = stefan_second_test(i, '1.2')
    
    L.append(fits)
    
reasonable_output = {'1.0': {'gaussian': {'mean': (0.9, 1.1)}},
 '1.2': {'gaussian': {'mean': (1.1, 1.3)}},
 '1.6': {'gaussian': {'mean': (1.5, 1.8)}},
 '2.0': {'gaussian': {'mean': (1.9, 2.2)}}}

for entry in L:
    if not entry:
        continue
    violate = False
    
    for channel in reasonable_output:
        for fit_type in reasonable_output[channel]:
            for param in reasonable_output[channel][fit_type]:
                
                low, high = reasonable_output[channel][fit_type][param]
                if not low < entry[channel][fit_type][param] < high:
                    print(channel, fit_type, param)
                    violate = True
    
    if violate:
        continue
    good += 1
    
print(100, good)
#len(L), len(filter_data(L))
# num_bad = 0
# num_bad2 = 0
# for i in range(len(data)):
#     fits = stefan_second_test(i, '1.2')
#     if not fits or not 1.2 < fits['1.2']['gaussian']['mean'] < 1.3:
#         num_bad += 1
#     if not data[i]['stefan_fit'] or not 1.2 < data[i]['stefan_fit']['1.2']['gaussian']['mean'] < 1.3:
#         num_bad2 += 1
#     if i%100==0:
#         print(num_bad, num_bad2, i)
# print(num_bad)
print(reasonable_output)
stefan_second_test(40293,'1.2')

1.6 gaussian mean
1.6 gaussian mean
1.6 gaussian mean
1.6 gaussian mean
1.6 gaussian mean
1.0 gaussian mean
1.6 gaussian mean
1.2 gaussian mean
1.6 gaussian mean
1.2 gaussian mean
1.6 gaussian mean
1.2 gaussian mean
1.2 gaussian mean
1.0 gaussian mean
1.2 gaussian mean
100 75
{'1.0': {'gaussian': {'mean': (0.9, 1.1)}}, '1.2': {'gaussian': {'mean': (1.1, 1.3)}}, '1.6': {'gaussian': {'mean': (1.5, 1.8)}}, '2.0': {'gaussian': {'mean': (1.9, 2.2)}}}


{'1.2': {'gaussian': {'mean': 1.2752755066599044,
   'sigma': 0.01816872731006732,
   'amp': 0.08176188929009903},
  'skew': {'A': -0.04807092570484599,
   'e': 0.011971725926982701,
   'w': -1.330262998920826,
   'a': 40.09370525489985,
   'peak_channel': 1.13024,
   'peak_value': 0.0,
   'peak_channel_inferred': 1.13024,
   'peak_value_inferred': 0.0},
  'raw_peak_channel': 1.27813,
  'raw_peak_value': 0.0852551884298572},
 '1.6': {'gaussian': {'mean': 1.5762844865664174,
   'sigma': 0.0224818073137273,
   'amp': 0.026633995296056052},
  'skew': {'A': 0.0015159861315809863,
   'e': 1.5997883768467882,
   'w': 0.03895195957901163,
   'a': -4.720226496435213,
   'peak_channel': 1.59018,
   'peak_value': 0.026443435056441953,
   'peak_channel_inferred': 1.5848891539153915,
   'peak_value_inferred': 0.02783803237340476},
  'raw_peak_channel': 1.59018,
  'raw_peak_value': 0.026427850055687832},
 '2.0': {'gaussian': {'mean': 2.027075352834752,
   'sigma': 0.02865816991946863,
   'amp': 0.0

In [11]:
reasonable_output

{'1.0': {'gaussian': {'mean': (0.9, 1.1)},
  'skew': {'peak_channel': (0.9, 1.1)}},
 '1.2': {'gaussian': {'mean': (1.1, 1.3)},
  'skew': {'peak_channel': (1.1, 1.3)}},
 '1.6': {'gaussian': {'mean': (1.5, 1.8)},
  'skew': {'peak_channel': (1.5, 1.8)}},
 '2.0': {'gaussian': {'mean': (1.9, 2.2)},
  'skew': {'peak_channel': (1.9, 2.2)}}}

### Time Bucketing and Restricting Viewing Geometry

Call time_bucket to bucket all entries into k (default 20) equally spaced time buckets. Can also use the function viewing_geometry to specify the range of incidence, emission, and phase angles. 

In [ ]:
buckets = time_bucket(filtered_data)
[(x, len(buckets[x])) for x in buckets]

In [ ]:
viewing_geometry

### Rainbow Plots

Using the data split into time buckets, plot one gaussian for each time bucket to achieve a rainbow-like effect.

![Two-micron rainbow](rainbow.png "Rainbow")

In [ ]:
def plot_average_fits_rainbow(buckets, channel = '2.0', fit_type = 'regular_fit'):
    ncolors = len(buckets)
    colors = plt.cm.nipy_spectral(np.linspace(0.05,0.95,ncolors))
    
    parameters = {x: {} for x in buckets}
    
    for i, bucket in enumerate(buckets):
        mean = []
        amplitudes = []
        width = []
        
        for entry in buckets[bucket]:
            amplitudes.append(entry[fit_type][channel]['gaussian']['amp'])
            width.append(entry[fit_type][channel]['gaussian']['sigma'])
            mean.append(entry[fit_type][channel]['gaussian']['mean'])
        
        median = len(mean) // 2
        
        parameters[bucket]['mean'] = mean[median]
        parameters[bucket]['width'] = width[median]
        parameters[bucket]['amplitudes'] = amplitudes[median]
        
    for i, bucket in enumerate(parameters):
        if parameters[bucket].get('mean', None) != None:
            mean_, sigma_, amp_ = parameters[bucket]['mean'], parameters[bucket]['width'], parameters[bucket]['amplitudes']
            x = np.linspace(0, 5, 1000)
            plt.plot(x, gaussian(x, mean_, .1, 1), color = colors[i], label = bucket)

    plt.legend()

In [ ]:
plot_average_fits_rainbow(buckets, channel = '1.6', fit_type = 'stefan-plaw_fit')
plot_average_fits_rainbow(buckets, channel = '2.0', fit_type = 'stefan-plaw_fit')
plot_average_fits_rainbow(buckets, channel = '1.0', fit_type = 'stefan-plaw_fit')
plot_average_fits_rainbow(buckets, channel = '1.2', fit_type = 'stefan-plaw_fit')


### Temporal Plots

Plot a multi-panel figure given a list of pixels displaying how best fit parameters change with time.

In [ ]:
def plot_temporal(data, channel = '2.0', sample_every = 100):
    
    fig, axs = plt.subplots(4,3)
    
    info = {'stefan_g': {x: [] for x in data[0]['stefan_fit'][channel]['gaussian']},
            'stefan_s': {x: [] for x in data[0]['stefan_fit'][channel]['skew']},
            'regular_g': {x: [] for x in data[0]['regular_fit'][channel]['gaussian']},
            'regular_s': {x: [] for x in data[0]['regular_fit'][channel]['skew']},
            'ettimes': [],
            'starttimes': [],
            'jtimes': [],
            'incidences': [],
            'emissions': [],
            'phases': [],
            'powerlaw': {'scales': [], 'exps': []}}
            
    for i,entry in enumerate(data):
        if i%sample_every == 0:
            for x in entry['stefan_fit'][channel]['gaussian']:
                info['stefan_g'][x].append(entry['stefan_fit'][channel]['gaussian'][x])

            for x in entry['stefan_fit'][channel]['skew']:
                info['stefan_s'][x].append(entry['stefan_fit'][channel]['skew'][x])

            for x in entry['regular_fit'][channel]['gaussian']:
                info['regular_g'][x].append(entry['regular_fit'][channel]['gaussian'][x])

            for x in entry['regular_fit'][channel]['skew']:
                info['regular_s'][x].append(entry['regular_fit'][channel]['skew'][x]) 

            info['ettimes'].append(entry['ettime'])
            info['starttimes'].append(entry['starttime'])#[:-19])
            info['incidences'].append(entry['incidence'])
            info['emissions'].append(entry['emission'])
            info['phases'].append(entry['phase'])
            info['jtimes'].append(Time(entry['starttime'], format = 'iso').jd)

            if 'powerlaw' in entry['regular_fit']:
                info['powerlaw']['scales'].append(entry['regular_fit']['powerlaw']['scale'])
                info['powerlaw']['exps'].append(entry['regular_fit']['powerlaw']['exponent'])
            else:
                info['powerlaw']['scales'].append(None)
                info['powerlaw']['exps'].append(None)

#     for i in range(1):
#         for j in range(2):
#             #continue
#             axs[i, j].xaxis.set_tick_params(rotation=30, labelsize=5)

#             axs[i, j].set_xticks(sorted(list(set(info['jtimes'][::100]))))
#             axs[i, j].set_xticklabels(sorted(list(set(info['starttimes'][::100]))))
    
    print(len(info['stefan_g']['mean']))
    axs[0, 0].xaxis.set_tick_params(rotation=30, labelsize=5)
    axs[0, 0].scatter(info['jtimes'], info['regular_g']['mean'], alpha = .3)
    axs[0, 0].set_xticks(sorted(list(set(info['jtimes'][::100]))))
    axs[0, 0].set_xticklabels(sorted(list(set(info['starttimes'][::100]))))
    axs[0, 0].set_title('Regular Gaussian Mean')
    p = np.polyfit(info['jtimes'], info['regular_g']['mean'], 1)
    fit = np.polyval(p, info['jtimes'])
    axs[0, 0].plot(info['jtimes'], fit, c = 'r')
    print("Regular Gaussian Mean", fit[0], fit[-1], p)
    
    axs[0, 1].scatter(info['jtimes'], info['regular_g']['sigma'], alpha = .3)
    axs[0, 1].set_title('Regular Gaussian Width')
    axs[0, 1].set_ylim([0,.2])
    
    
    axs[0, 2].scatter(info['jtimes'], info['regular_g']['amp'], alpha = .3)
    axs[0, 2].set_title('Regular Gaussian Amplitude')
    
    axs[1, 0].scatter(info['jtimes'], info['regular_s']['peak_channel_inferred'], alpha = .3)
    axs[1, 0].set_title('Regular Skew Peak Channel')
    p = np.polyfit(info['jtimes'], info['regular_s']['peak_channel_inferred'], 1)
    fit = np.polyval(p, info['jtimes'])
    axs[1, 0].plot(info['jtimes'], fit, c = 'r')
    print("Regular Skew Peak Channel", fit[0], fit[-1], p)
    
    axs[1, 1].scatter(info['jtimes'], info['regular_s']['peak_value_inferred'], alpha = .3)
    axs[1, 1].set_title('Regular Skew Peak Value')
    
    axs[1, 2].scatter(info['jtimes'], info['powerlaw']['scales'], alpha = .3)
    axs[1, 2].set_title('Powerlaw Scale')
    axs[1, 2].set_ylim(-.5, .2)
    
    axs[2, 0].scatter(info['jtimes'], info['powerlaw']['exps'], alpha = .3)
    axs[2, 0].set_title('Powerlaw Exponent')
    axs[2, 0].set_ylim(-5,0)
    
    axs[2, 1].scatter(info['jtimes'], info['stefan_g']['mean'], alpha = .3)
    axs[2, 1].set_title('Stefan Gaussian Mean')
    p = np.polyfit(info['jtimes'], info['stefan_g']['mean'], 1)
    fit = np.polyval(p, info['jtimes'])
    axs[2, 1].plot(info['jtimes'], fit, c = 'r')
    print("Stefan Gaussian Mean", fit[0], fit[-1], p)
    
    axs[2, 2].scatter(info['jtimes'], info['stefan_g']['sigma'], alpha = .3)
    axs[2, 2].set_title('Stefan Gaussian Width')
    axs[2, 2].set_ylim([0,0.2])
    
    axs[3, 0].scatter(info['jtimes'], info['stefan_g']['amp'], alpha = .3)
    axs[3, 0].set_title('Stefan Gaussian Amplitude')
    axs[3, 0].set_ylim([0,1])
    
    axs[3, 1].scatter(info['jtimes'], info['stefan_s']['peak_channel_inferred'], alpha = .3)
    axs[3, 1].set_title('Stefan Skew Peak Channel')
    p = np.polyfit(info['jtimes'], info['stefan_s']['peak_channel'], 1)
    fit = np.polyval(p, info['jtimes'])
    axs[3, 1].plot(info['jtimes'], fit, c = 'r')
    print("Stefan Skew Peak Channel", fit[0], fit[-1], p)
    
    axs[3, 2].scatter(info['jtimes'], info['stefan_s']['peak_value_inferred'], alpha = .3)
    axs[3, 2].set_title('Stefan Skew Peak Value')
    axs[3, 2].set_ylim([0, .25])
    plt.tight_layout(pad=0.00001)
    
plot_temporal(filtered_data)
    

### Slopes for all (incidence, emission) bins

Bin incidence and emission into bins of range 10 each and find linear fit for each bin.

In [ ]:
min(data, key = lambda x: x['incidence'])['incidence'], max(data, key = lambda x: x['incidence'])['incidence']

In [ ]:
min(data, key = lambda x: x['emission'])['emission'], max(data, key = lambda x: x['emission'])['emission']

In [ ]:
def assign_bins(data, spread):
    bins = []
    for i in range(90//spread):
        current = [((spread*i),(spread*(i+1)))]
        for j in range(90//spread):
            bins.append(current + [(spread*j,spread*(j+1))])
    
    assignment = {tuple(x): [] for x in bins}
    for entry in data:
        for _bin in bins:
            if _bin[0][0] <= entry['incidence'] <= _bin[0][1] and _bin[1][0] <= entry['emission'] <= _bin[1][1]:
                assignment[tuple(_bin)].append(entry)
    return assignment

def analyze_bins(data, spread, channel, fit_type, function, mean_field):#:, weighted = False):
    
    binned_data = assign_bins(data, spread)
    
    bin_shifts = {b: None for b in binned_data}
    
    for bin in binned_data:
        if not binned_data[bin] or len(binned_data[bin]) < 1000:
            print(f"Skipping bin {bin} due to little/no values being present.")
            continue
        print(f"Bin {bin} has length {len(binned_data[bin])}")
        times = [Time(entry['starttime'], format = 'iso').jd for entry in binned_data[bin]]
        means = [entry[fit_type][channel][function][mean_field] for entry in binned_data[bin]]
        
        
        p = np.polyfit(times, means, 1)
        fit = np.polyval(p, times)
        
        bin_shifts[bin] = [fit[0], fit[-1], p]
    
    #if not weighted:
    print("MEAN", np.mean([x[-1][0] for x in bin_shifts.values() if x != None]))
    #print("MEDIAN", np.median([x[-1][0] for x in bin_shifts.values() if x != None]))
    #else:
    print("WEIGHTED MEAN", np.average([bin_shifts[x][-1][0] for x in bin_shifts if bin_shifts[x]!= None], weights = [len(binned_data[x]) for x in bin_shifts if bin_shifts[x] != None]))
    print("MEDIAN", np.median([x[-1][0] for x in bin_shifts.values() if x != None]))
    print("STDEV", np.std([x[-1][0] for x in bin_shifts.values() if x!= None]))
    
    
    return bin_shifts

In [ ]:
analyze_bins(filtered_data, 10, '2.0', 'regular_fit', 'skew', 'peak_channel_inferred')#, False)

```
results from previous: (mean, weighted mean, median)


analyze_bins(filtered_data, 90, '2.0', 'regular_fit', 'gaussian', 'mean') => -3.77e-06
analyze_bins(filtered_data, 90, '2.0', 'stefan_fit', 'gaussian', 'mean') => -3.93e-06

analyze_bins(filtered_data, 10, '2.0', 'stefan_fit', 'gaussian', 'mean') => -2.85e-06, -2.82e-06, -2.79e-06
analyze_bins(filtered_data, 10, '1.6', 'stefan_fit', 'gaussian', 'mean') => -3.17e-06, -2.99e-06, -2.99e-06
analyze_bins(filtered_data, 10, '1.2', 'stefan_fit', 'gaussian', 'mean') => -3.14e-06, -3.11e-06, -3.01e-06
analyze_bins(filtered_data, 10, '1.0', 'stefan_fit', 'gaussian', 'mean') => -2.34e-06, -2.38e-06, -2.37e-06

analyze_bins(filtered_data, 10, '2.0', 'regular_fit', 'gaussian', 'mean') => -2.94e-06, -2.96e-06, -2.95e-06
analyze_bins(filtered_data, 10, '1.6', 'regular_fit', 'gaussian', 'mean') => -3.21e-06, -3.10e-06, -3.13e-06
analyze_bins(filtered_data, 10, '1.2', 'regular_fit', 'gaussian', 'mean') => -2.55e-06, -2.70e-06, -2.65e-06
analyze_bins(filtered_data, 10, '1.0', 'regular_fit', 'gaussian', 'mean') => -2.41e-06, , -2.58e-06, -2.65e-06

analyze_bins(filtered_data, 10, '1.0', stefan-plaw_fit', 'gaussian', 'mean') => -2.42e-06, -2.58e-06, -2.65e-06
analyze_bins(filtered_data, 10, '1.2', stefan-plaw_fit', 'gaussian', 'mean') => -2.55e-06, -2.70e-06, -2.65e-06
analyze_bins(filtered_data, 10, '1.6', stefan-plaw_fit', 'gaussian', 'mean') => -3.21e-06, -3.10e-06, -3.13e-06
analyze_bins(filtered_data, 10, '2.0', stefan-plaw_fit', 'gaussian', 'mean') => -2.94e-06, -2.96e-06, -2.95e-06

skew weird because discrete values
```

In [ ]:
data[0]

In [ ]:
def scatter_term_estimate(entry, channel, stefan = False):
    
    amp = entry['stefan_spectrum'][channel] if stefan else entry['spectrum'][channel]
    
    return amp * 4*(np.cos(np.radians(entry['incidence'])) + np.cos(np.radians(entry['emission'])))/(P(np.radians(entry['phase'])) * np.cos(np.radians(entry['incidence'])))
    

In [ ]:
[scatter_term_estimate(data[0], x) for x in range(255)]

In [ ]:
s = data[0]['spectrum']
s = s.tolist()
plt.plot(vims_wave, s, label = 'old')
plt.plot(vims_wave, s[:5]+[1.15*x for x in s[5:15]]+s[15:19]+[x*1.5 for x in s[19:32]] + s[32:40] + [1.6*x for x in s[40:50]] + s[50:65] + [x*1.29 for x in s[65:75]] + s[75:])

plt.legend()

In [ ]:
np.array([1,2,3]) + np.array([3,4,2,1])